# SMB Lateral Movement Burst

**High number of SMB connections to distinct hosts in a short time window.**

This notebook is a **ready-to-use SOC playbook** with:

- Triage checklist
- Splunk SPL queries
- Microsoft Sentinel KQL queries
- Elastic KQL/EQL concepts
- Containment & response commands (PowerShell / Windows / MDE)

> Replace placeholders like `<USER>`, `<HOST>`, `<DEVICE_ID>`, `<TENANT_DOMAIN>` before running.


## Triage Checklist

**Collect:** user/account, host/device, source IP, time window, impacted resources, and any correlated alerts.

**Validate:** baseline/allowlists (VPN egress, jump hosts, admin tools) before escalating.

**Preserve:** keep a copy of key logs and process trees before remediation when possible.

Focus: **source host**, **target count**, **account used**, **remote exec indicators (PsExec/WMI/WinRM)**.


## Investigation Queries

Measure distinct SMB targets and pivot to process events for remote execution tools.


In [ ]:
// KQL
let window=10m;
DeviceNetworkEvents
| where TimeGenerated > ago(7d)
| where DeviceName == '<HOST>'
| where RemotePort == 445
| summarize Targets=dcount(RemoteIP), TargetList=make_set(RemoteIP) by InitiatingProcessAccountName, bin(TimeGenerated, window)
| order by Targets desc


In [ ]:
# Splunk SPL
index=network dest_port=445 src_ip=<SRC_IP>
| bucket _time span=10m
| stats dc(dest_ip) as targets values(dest_ip) as target_list by user _time
| where targets>=10


In [ ]:
# Elastic (KQL/EQL)
event.category:network AND destination.port:445


## Containment & Response

Isolate source host, block SMB egress from that segment temporarily, and reset compromised creds.


In [ ]:
# PowerShell
## Block SMB outbound from a host (Windows Firewall)
New-NetFirewallRule -DisplayName "TEMP Block SMB Out" -Direction Outbound -Protocol TCP -RemotePort 445 -Action Block


In [ ]:
# PowerShell
## Reset user password / rotate admin credentials (org-specific)
# Use your IAM tooling / AD commands.
